In [ ]:
!pip install timm

### 함수

In [ ]:
# ===== 1) Standard Library =====
import os
import time
import random
from collections import Counter

# ===== 2) Third-Party Core =====
import numpy as np
import pandas as pd

# ===== 3) PyTorch Core =====
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR  # (1) 여기서 한 번 임포트

# ===== 4) TorchVision / Data Loading =====
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

# ===== 5) Albumentations =====
import albumentations as A
from albumentations.pytorch import ToTensorV2
import albumentations.pytorch as A_torch  # A_torch.ToTensorV2()도 사용 가능

# ===== 6) Models (timm) =====
import timm

# ===== 7) Metrics / CV (scikit-learn) =====
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix

# ===== 8) Vision / Utility =====
from PIL import Image
from tqdm import tqdm

# ===== 9) Visualization =====
import matplotlib.pyplot as plt
import seaborn as sns


# TTA를 위한 보조 함수
def _orientation_views(batch_t: torch.Tensor):
    # batch_t: [B, C, H, W]
    outs = []
    # 0) original
    outs.append(batch_t)
    # 1) horizontal flip
    outs.append(torch.flip(batch_t, dims=[3]))
    # 2) 180°
    outs.append(torch.rot90(batch_t, k=2, dims=[2, 3]))
    # 3) 180° + hflip
    o = torch.rot90(batch_t, k=2, dims=[2, 3])
    outs.append(torch.flip(o, dims=[3]))
    return outs

# TTA 예측 평균 함수
def tta_mean_logits(model, batch_t: torch.Tensor, device, use_cuda: bool):
    logits_list = []
    for view in _orientation_views(batch_t):
        with torch.amp.autocast(device_type='cuda', enabled=use_cuda):
            logits = model(view)
        logits_list.append(logits)
    # [4, B, C] -> [B, C]
    return torch.stack(logits_list, dim=0).mean(dim=0)

# 시드 고정
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False     
set_seed(42)


# device/AMP 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

use_cuda = torch.cuda.is_available()
scaler = torch.cuda.amp.GradScaler(enabled=use_cuda)

### 기존 데이터 분리

In [ ]:
# --- K-Fold 설정 ---
N_SPLITS = 5 
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

df = pd.read_csv("/root/cv_project/datasets/data/train.csv")

print(f"✅ K-Fold: Using {N_SPLITS} splits on {len(df)} total images.")

### 가중치 계산

In [ ]:
# --- 클래스별 개수 계산 ---
class_counts = Counter(df["target"].tolist())
num_classes = df["target"].nunique()
total_samples = len(df)

# --- 클래스별 가중치 계산 (정규화된 역빈도 방식) ---
freq = np.array([class_counts[i] for i in range(num_classes)], dtype=np.float32)
inv_freq = 1.0 / np.maximum(freq, 1.0)  # 0 나눗셈 방지
weights = inv_freq * (num_classes / inv_freq.sum())  # 합 = 클래스 수로 정규화

# --- 텐서로 변환 후 GPU로 이동 ---
class_weights = torch.tensor(weights, dtype=torch.float, device=device)
print(f"✅ Class weights for WeightedLoss calculated: {np.round(weights, 4)}")


### 클래스 정의

In [ ]:
# 데이터셋 클래스를 정의
class ImageDataset(Dataset):
    def __init__(self, df, path, transform=None): 
        self.path = path
        self.transform = transform
        
        print("Checking data integrity...")
        name_col = df.columns[0]     # ID
        target_col = df.columns[1]   # target
        
        clean_data = []
        # 존재하는 이미지만 필터링
        for index, row in tqdm(df.iterrows(), total=len(df), desc="Cleaning CSV"):
            file_path = os.path.join(self.path, row[name_col])
            if os.path.exists(file_path):
                clean_data.append((row[name_col], row[target_col]))
        
        self.df = clean_data 
        print(f"Original: {len(df)}, Found: {len(self.df)}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx] 
        
        # 이미지를 RGB로 강제 변환(흑백이미지 대비)
        img = np.array(Image.open(os.path.join(self.path, name)).convert('RGB')) 
        
        # Augmentaions 변환 적용
        if self.transform:
            img = self.transform(image=img)['image']
        
        # 👈 target을 int가 아닌 long tensor로 직접 변환 (가장 중요)
        # loss_fn(preds, targets)는 targets가 LongTensor이기를 기대합니다.
        return img, torch.tensor(target, dtype=torch.long)

### epoch 계산 함수

In [ ]:
# 학습용 함수 (AMP 안정화 적용)
def train_one_epoch(loader, model, optimizer, loss_fn, device, scaler):
    model.train()
    train_loss = 0.0
    preds_list, targets_list = [], []

    pbar = tqdm(loader, leave=False)
    for image, targets in pbar:
        
        # GPU전송 최적화
        image = image.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        model.zero_grad(set_to_none=True)

        # AMP: CUDA일 때만 활성화
        with torch.amp.autocast(device_type='cuda', enabled=use_cuda):
            preds = model(image)
            loss = loss_fn(preds, targets)
        
        # Gradient 계산 및 파라미터 갱신
        if use_cuda:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())
        pbar.set_description(f"Loss: {loss.item():.4f}")

    # 통계값 기록
    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')
    return {"train_loss": train_loss, "train_acc": train_acc, "train_f1": train_f1}


# 검증용 함수 (AMP 안정화 적용)
@torch.no_grad()   # Gradient 계산 비활성화
def validate_one_epoch(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0.0
    preds_list, targets_list = [], []

    for image, targets in loader:
        image = image.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        # AMP: CUDA일 때만 활성화
        with torch.amp.autocast(device_type='cuda', enabled=use_cuda):
            preds = model(image)
            loss = loss_fn(preds, targets)

        val_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')
    return {"val_loss": val_loss, "val_acc": val_acc, "val_f1": val_f1}



### 모델/하이퍼파라미터 선택

In [ ]:
# data config
data_path = '/root/cv_project/datasets/data'

# model config
model_name = 'swin_base_patch4_window12_384' 

# training config
img_size = 384  # 수정된 부분
LR = 5e-5
EPOCHS = 15
BATCH_SIZE = 8
num_workers = 4
PATIENCE = 3

### 이미지 조정

In [ ]:
# 학습용 이미지 변환
trn_transform = A.Compose([
    A.LongestMaxSize(max_size=img_size), # 384에 맞춤
    A.PadIfNeeded(
        min_height=img_size, 
        min_width=img_size, 
        border_mode=0,   # 0 = cv2.BORDER_CONSTANT (고정색)
        value=255        # 패딩색상 (흰색)
    ), 
    # ------------------------------------------------
    
    # 기하 변환: 과하지 않게, 드물게 강한 방향만 섞기
    A.ShiftScaleRotate(
        shift_limit=0.02,  # 위치 이동 2%
        scale_limit=0.05,  # 크기 5%
        rotate_limit=5,    # 회전 5도
        p=0.5              # 확률 50%
    ),
    A.Perspective(scale=(0.02, 0.07), p=0.25),   # 약한 원근 왜곡
    A.RandomRotate90(p=0.10),                    # 90/180/270 중 하나(10% 확률)
    
    # 좌우/상하 반전
    A.HorizontalFlip(p=0.25),
    A.VerticalFlip(p=0.08),
    
    # 선명도/대비
    A.OneOf([
        A.Sharpen(alpha=(0.1, 0.3), lightness=(0.9, 1.1), p=1.0),   # 문서 가장자리
        A.CLAHE(clip_limit=(2, 4), tile_grid_size=(8, 8), p=1.0),   # 밝기 보정
    ], p=0.45),

    # 밝기/대비 변화 -> 스캐너 노이즈, 그림자 차이
    A.RandomBrightnessContrast(brightness_limit=0.12, contrast_limit=0.12, p=0.4),
    A.ToGray(p=0.25),

    # 정규화 + 텐서 변환
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A_torch.ToTensorV2(),
])

# 테스트용 이미지 변환 
tst_transform = A.Compose([
    A.LongestMaxSize(max_size=img_size), 
    A.PadIfNeeded(
        min_height=img_size, 
        min_width=img_size, 
        border_mode=0, 
        value=255
    ),
    # 테스트용은 Augmentation 제거
    
    # 정규화 + 텐서 변환
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A_torch.ToTensorV2(),
])

### dataloader

In [ ]:
# 추론용 테스트 이미지 구성
tst_df = pd.read_csv("/root/cv_project/datasets/data/sample_submission.csv")
tst_dataset = ImageDataset(
    tst_df, 
    "/root/cv_project/datasets/data/test/",
    transform=tst_transform
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

### k-fold 학습 루프

In [ ]:
# --- K-Fold 학습 루프 시작 ---

all_fold_val_f1 = [] # 모든 Fold의 검증 F1 스코어를 저장

for fold, (trn_idx, val_idx) in enumerate(skf.split(df, df['target'])):
    print(f"\n--- FOLD {fold + 1}/{N_SPLITS} START ---")

    # 1. Fold별로 데이터 분리
    trn_df = df.iloc[trn_idx]
    val_df = df.iloc[val_idx]

    # 2. Fold별로 Dataset 및 DataLoader 생성
    trn_dataset = ImageDataset(trn_df, "/root/cv_project/datasets/data/train/", transform=trn_transform)
    val_dataset = ImageDataset(val_df, "/root/cv_project/datasets/data/train/", transform=tst_transform)

    trn_loader = DataLoader(
        trn_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    # 3. 매 Fold마다 모델, 옵티마이저, 스케줄러 새로 초기화
    model = timm.create_model(
        model_name,
        pretrained=True,
        num_classes=17
    ).to(device)
    loss_fn = nn.CrossEntropyLoss(weight=class_weights,label_smoothing=0.1)
    optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
    
    
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)

    best_f1 = -1
    best_epoch = 0
    epochs_no_improve = 0  

    # 4. Epoch 학습 루프
    for epoch in range(EPOCHS):
        tr = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device, scaler=scaler)
        va = validate_one_epoch(val_loader, model, loss_fn, device=device)

        print(f"[Fold {fold+1}/Epoch {epoch}] Train F1: {tr['train_f1']:.4f} | Val F1: {va['val_f1']:.4f}")

        # 베스트 모델 갱신 (f1기준)
        if va["val_f1"] > best_f1:
            best_f1 = va["val_f1"]
            best_epoch = epoch
            torch.save(model.state_dict(), f"output/best_model_fold{fold}.pt") 
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        scheduler.step()

        # 조기 종료
        if epochs_no_improve >= PATIENCE:
            print(f"--- Early Stopping at Epoch {epoch} as Val_F1 did not improve for {PATIENCE} epochs ---")
            break

    print(f"✅ Fold {fold+1} Best Val_F1: {best_f1:.4f} (epoch {best_epoch})")
    all_fold_val_f1.append(best_f1)

print(f"\n--- K-Fold Training Finished ---")
print(f"✅ All Fold Best F1 Scores: {all_fold_val_f1}")
mean_f1 = np.mean(all_fold_val_f1)
std_f1 = np.std(all_fold_val_f1)

# 평균과 표준편차 출력
print(f"✅ Final CV Score (Mean ± Std): {mean_f1:.4f} ± {std_f1:.4f}")

### oof 예측

In [ ]:
# ── OOF(Out-Of-Fold) 예측 생성 ──
print(f"\n--- Generating Out-of-Fold (OOF) Predictions ---")

df['oof_preds'] = -1
meta_df = pd.read_csv("/root/cv_project/datasets/data/meta.csv")
class_name_map = dict(zip(meta_df['target'], meta_df['class_name']))

for fold, (trn_idx, val_idx) in enumerate(skf.split(df, df['target'])):
    print(f"Generating OOF for Fold {fold + 1}/{N_SPLITS}")

    val_df = df.iloc[val_idx]
    val_dataset = ImageDataset(
        val_df,
        "/root/cv_project/datasets/data/train/",
        transform=tst_transform
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    best_model_path = f"output/best_model_fold{fold}.pt"
    if os.path.exists(best_model_path):
        model = timm.create_model(model_name, pretrained=False, num_classes=17).to(device)
        model.load_state_dict(torch.load(best_model_path, map_location=device))
        model.eval()

        fold_preds = []
        with torch.inference_mode():
            for image, _ in tqdm(val_loader, desc=f"Generating OOF Fold {fold + 1}"):
                image = image.to(device, non_blocking=True)
                # ★ TTA 제거: 그대로 forward 후 argmax
                logits = model(image)
                pred = logits.argmax(dim=1).detach().cpu().numpy()
                fold_preds.extend(pred)

        assert len(fold_preds) == len(val_df), "OOF length mismatch"
        df.loc[val_df.index, 'oof_preds'] = fold_preds
    else:
        print(f"⚠️ Model for Fold {fold} not found!")

df['oof_preds'] = df['oof_preds'].astype(int)
print("✅ OOF Prediction generation finished.")


df['oof_preds'] = df['oof_preds'].astype(int)

# 분석용 DF
analyzable_df = df[df['oof_preds'] != -1].copy()
analyzable_df['target_name'] = analyzable_df['target'].map(class_name_map)
analyzable_df['oof_preds_name'] = analyzable_df['oof_preds'].map(class_name_map)

print("\n--- Top 10 Misclassified Classes (Based on True Label) ---")
print(
    analyzable_df[analyzable_df['target'] != analyzable_df['oof_preds']]
    ['target_name'].value_counts().head(10)
)

print("\n--- Generating Confusion Matrix ---")
ordered_targets = sorted(meta_df['target'].unique().tolist())
class_labels = [class_name_map[i] for i in ordered_targets]

cm = confusion_matrix(
    analyzable_df['target'],
    analyzable_df['oof_preds'],
    labels=ordered_targets
)
cm_sum = np.nansum(cm, axis=1)[:, np.newaxis]
cm_normalized = np.divide(
    cm.astype('float'), cm_sum,
    out=np.zeros_like(cm, dtype=float),
    where=(cm_sum != 0)
)

plt.figure(figsize=(18, 14))
sns.heatmap(
    cm_normalized, annot=True, fmt=".2f", cmap="Blues",
    xticklabels=class_labels, yticklabels=class_labels
)
plt.title("OOF Confusion Matrix (Normalized by True Label)", fontsize=16)
plt.ylabel("True Label (실제 정답)", fontsize=12)
plt.xlabel("Predicted Label (모델 예측)", fontsize=12)
plt.show()


### k-fold 앙상블

In [ ]:
# ── K-Fold 앙상블 추론 ──
print(f"\n--- Starting K-Fold Ensemble Inference ---")

all_fold_logits = []  # 각 fold의 전체 테스트 로짓 [N, C]

for fold in range(N_SPLITS):
    best_model_path = f"output/best_model_fold{fold}.pt"
    if os.path.exists(best_model_path):
        print(f"✅ Loading model from Fold {fold}: {best_model_path}")
        model = timm.create_model(model_name, pretrained=False, num_classes=17).to(device)
        model.load_state_dict(torch.load(best_model_path, map_location=device))
        model.eval()

        fold_logits = []
        with torch.inference_mode():
            for image, _ in tqdm(tst_loader, desc=f"Predicting Fold {fold} (no TTA)"):
                image = image.to(device, non_blocking=True)
                with torch.amp.autocast(device_type='cuda', enabled=torch.cuda.is_available()):
                    logits = model(image)  # [B, C]
                fold_logits.append(logits.detach().cpu().numpy())

        fold_logits = np.concatenate(fold_logits, axis=0)  # [N, C]
        all_fold_logits.append(fold_logits)
    else:
        print(f"⚠️ Model for Fold {fold} not found!")

# Fold 평균 (logit 평균) → 확률 → 예측
avg_logits = np.mean(all_fold_logits, axis=0)                 # [N, C]
# torch 없이 numpy softmax
exp = np.exp(avg_logits - np.max(avg_logits, axis=1, keepdims=True))
avg_probs = exp / np.sum(exp, axis=1, keepdims=True)          # [N, C]

# 저장 파일
np.save("output/swin_logits.npy", avg_logits)
np.save("output/swin_probs.npy",  avg_probs)

final_preds = np.argmax(avg_probs, axis=1)

In [ ]:
# --- 제출 파일 생성 ---
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = final_preds # 최종 앙상블 예측 결과

sample_submission_df = pd.read_csv("/root/cv_project/datasets/data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

pred_df.to_csv("output/pred_14.csv", index=False) # 새 이름으로 저장
print("\n✅ K-Fold ensemble prediction saved to 'output/pred_kfold.csv'")

### vit 모델